In [3]:
import sys
sys.path.append("/home/mat/repos/gpu_monitor") 

In [4]:
from gpumon import nvidia_run, extract_and_print, extract
import asyncio

from bokeh.models import ColumnDataSource, TapTool, CustomJS, HoverTool
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook

%load_ext autoreload
%autoreload 2

In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
sample_data = [extract(nvidia_run()) for i in range(10)]

In [7]:
from itertools import count
import pandas as pd

In [8]:
def t_gen(sample_data):
    for msg in sample_data:
        yield {'gpu {}'.format(i):gpu['temperature']['gpu_temp'] for gpu, i in zip(msg['gpu'], count())}

In [9]:
df = pd.DataFrame(list(t_gen(sample_data)))

In [10]:
df = df.stack().reset_index(level=1).rename(columns={'level_1':'gpu',0:'temp'})

In [11]:
df = df.assign(temp=df['temp'].str.split().apply(lambda x: int(x[0])))

In [12]:
df = pd.DataFrame(list(t_gen(sample_data)))

In [14]:
df['gpu 0'] = df['gpu 0'].str.split().apply(lambda x: int(x[0]))
df['gpu 1'] = df['gpu 1'].str.split().apply(lambda x: int(x[0]))
df['gpu 2'] = df['gpu 2'].str.split().apply(lambda x: int(x[0]))
df['gpu 3'] = df['gpu 3'].str.split().apply(lambda x: int(x[0]))

In [15]:
df.head()

,gpu 0,gpu 1,gpu 2,gpu 3
0,32,29,34,36
1,32,29,34,36
2,32,29,34,36
3,32,29,34,36
4,32,29,34,36


In [16]:
df.reset_index().dtypes

index    int64
gpu 0    int64
gpu 1    int64
gpu 2    int64
gpu 3    int64
dtype: object

In [17]:
from bokeh.palettes import Paired

In [85]:
data_dict = {'gpu {}'.format(gpu):[] for gpu in range(4)}
data_dict['index'] = []
data = ColumnDataSource(data=data_dict)

In [42]:
def gpu_temperature_plot(num_gpus=4, plot_width=600, plot_height=400, y_range=(0, 80)):
#     data = ColumnDataSource(data=df.reset_index())
    p = figure(plot_width=plot_width, plot_height=plot_height, y_range=y_range)
    for gpu, color in zip(range(num_gpus), Paired[12]) :
        p.line('index', 
               'gpu {}'.format(gpu), 
               line_width=4, 
               source=data, 
               color=color, 
               legend="GPU {}".format(gpu))
    return p

In [86]:
def gpu_util_plot(num_gpus=4, plot_width=600, plot_height=400, y_range=(0, 110)):
    p = figure(plot_width=plot_width, plot_height=plot_height, y_range=y_range)
    for gpu, color in zip(range(num_gpus), Paired[12]) :
        p.line('index', 
               'gpu {}'.format(gpu), 
               line_width=4, 
               source=data, 
               color=color, 
               legend="GPU {}".format(gpu))
    return p

In [87]:
handle = show(gpu_util_plot(), notebook_handle=True)

In [88]:
a=extract(nvidia_run())

In [89]:
def convert_temp_string(t_string):
    return int(t_string.split()[0])

In [90]:
def extract_temperature(msg):
    return {'gpu {}'.format(i):[convert_temp_string(gpu['temperature']['gpu_temp'])] for gpu, i in zip(msg['gpu'], count())}

In [91]:
def extract_utilization(msg):
    return {'gpu {}'.format(i):[convert_temp_string(gpu['utilization']['gpu_util'])] for gpu, i in zip(msg['gpu'], count())}

In [92]:
from toolz import pipe

In [93]:
pipe(nvidia_run(), extract)

{'gpu': [{'temperature': OrderedDict([('gpu_temp', '50 C'),
                ('gpu_temp_max_threshold', '91 C'),
                ('gpu_temp_slow_threshold', '88 C')]),
   'utilization': OrderedDict([('gpu_util', '77 %'),
                ('memory_util', '50 %'),
                ('encoder_util', '0 %'),
                ('decoder_util', '0 %')])},
  {'temperature': OrderedDict([('gpu_temp', '35 C'),
                ('gpu_temp_max_threshold', '91 C'),
                ('gpu_temp_slow_threshold', '88 C')]),
   'utilization': OrderedDict([('gpu_util', '0 %'),
                ('memory_util', '0 %'),
                ('encoder_util', '0 %'),
                ('decoder_util', '0 %')])},
  {'temperature': OrderedDict([('gpu_temp', '41 C'),
                ('gpu_temp_max_threshold', '91 C'),
                ('gpu_temp_slow_threshold', '88 C')]),
   'utilization': OrderedDict([('gpu_util', '0 %'),
                ('memory_util', '0 %'),
                ('encoder_util', '0 %'),
                ('decode

In [56]:
for i in range(100):
    new_data = pipe(nvidia_run(), extract, extract_temperature)
    new_data['index']=[i]
    data.stream(new_data, rollover=20, setter=None)
    push_notebook(handle=handle)

In [94]:
for i in range(100):
    new_data = pipe(nvidia_run(), extract, extract_utilization)
    new_data['index']=[i]
    data.stream(new_data, rollover=None, setter=None)
    push_notebook(handle=handle)